In [1]:
from pprint import pprint as pp
import gc #garbage collector usado no createinvertedindex

In [2]:
import gensim.models.keyedvectors as word2vec
from gensim.models import KeyedVectors

def loadWordEmbeddingsModel(filename = "word_embeddings/word2vec/GoogleNews-vectors-negative300.bin"):
    model = KeyedVectors.load_word2vec_format(filename,
                                                       binary=True, limit=500000)
    return model

model = loadWordEmbeddingsModel()

In [3]:
class BabelItemsIter:
    def __init__(self,babelhash):
        __slots__ = ('__babelhash')
        self.__babelhash = babelhash    
        
    def __len__(self):
        return len(self.__babelhash)
    
    def __contains__(self,item):
        (key,value) = item
        return key in self.__babelhash and self.__babelhash[key]==value
        
    def __iter__(self):
        for key in self.__babelhash.keys():
            yield key, self.__babelhash[key]
            
    #Apesar de que segundo o PEP 3106 (https://www.python.org/dev/peps/pep-3106/) recomenda que façamos
    # outros métodos, como and,eq,ne para permitir que a saída seja um set,
    # não estamos preocupados com isso aqui.

In [4]:
class BabelHash(dict):
    
    def __init__(self,babel={}):
        __slots__ = ('__babel')
        dict.__init__(self)
        self.__babel = babel
        
    def __getidfromkey__(self,key):
        return self.__babel[key]
    
    def __getkeyfromid__(self,key_id):
        key = self.__babel[key_id]
        return key
    
    def __getitem__(self,key):
        key_id = self.__getidfromkey__(key)
        return dict.__getitem__(self,key_id)
    
    def __setitem__(self,key,value):    
        try:
            key_id = self.__babel[key]
        except KeyError:
            key_id = len(self.__babel)+1
                     
            self.__babel[key] = key_id
            self.__babel[key_id] = key
        
        dict.__setitem__(self, key_id,value)
    
    def __delitem__(self, key):
        key_id = self.__getidfromkey__(key)
        dict.__delitem__(self, key_id)
        
    def __missing__(self,key):
        key_id = self.__getidfromkey__(key)
        return key_id
        
    def __delitem__(self, key):
        key_id = self.__getidfromkey__(key)
        dict.__delitem__(self,key_id)
    
    def __contains__(self, key):
        try:
            key_id = self.__getidfromkey__(key)
        except KeyError:
            return False
        
        return dict.__contains__(self,key_id)    
    
    def __iter__(self):
        for key_id in dict.keys(self):
            yield self.__getkeyfromid__(key_id)
    
    def keys(self):
        for key_id in dict.keys(self):
            yield self.__getkeyfromid__(key_id)
    
    def items(self):
        return BabelItemsIter(self)
    
    def get(self,key):
        value = None
        if key in self:
            value = self.__getitem__(key)
        return value
    
    def setdefault(self,key,default=None):
        if key not in self:
            self[key]=default
        return self[key]
    
    def printBabel(self):
        print(self.__babel)

In [5]:
class WordHash(dict):      
        
    def __init__(self,*args): 
        dict.__init__(self,args)
    
    def addMapping(self,word,table,attribute,ctid):
        self.setdefault( word, (0, BabelHash() ) )                    
        self[word].setdefault(table , BabelHash() )       
        self[word][table].setdefault( attribute , [] ).append(ctid)
        
    def getMappings(self,word,table,attribute):
        return self[word][table][attribute]
    
    def getIAF(self,key):
        return dict.__getitem__(self,key)[0]
    
    def setIAF(self,key,IAF):
        
        oldIAF,oldValue = dict.__getitem__(self,key)
        
        dict.__setitem__(self, key,  (IAF,oldValue)  )
    
    def __getitem__(self,word):
        return dict.__getitem__(self,word)[1]
    
    def __setitem__(self,word,value): 
        oldIAF,oldValue = dict.__getitem__(self,word)
        dict.__setitem__(self, word,  (oldIAF,value)  )

In [6]:
import psycopg2
from psycopg2 import sql
import string

import nltk 
from nltk.corpus import stopwords
nltk.download('stopwords')

stw_set = set(stopwords.words('english')) - {'will'}

class DatabaseIter:
    def __init__(self,embeddingModel,dbname='dblp',user='imdb',password='imdb'):
        self.dbname=dbname
        self.user=user
        self.password =password
        self.embeddingModel=embeddingModel

    def __iter__(self):
        with psycopg2.connect(dbname=self.dbname,user=self.user,password=self.password) as conn:
            with conn.cursor() as cur:

                # Get list of tablenames

                GET_TABLE_NAMES_SQL='''
                    SELECT DISTINCT table_name
                    FROM information_schema.columns 
                    WHERE table_schema='public';
                ''' 
                cur.execute(GET_TABLE_NAMES_SQL)

                for table in cur.fetchall():
                    table_name = table[0]

                    if table_name not in self.embeddingModel:
                        print('TABLE ',table_name, 'SKIPPED')
                        continue

                    print('INDEXING TABLE ',table_name)

                    #Get all tuples for this tablename
                    cur.execute(
                        sql.SQL("SELECT ctid, * FROM {};").format(sql.Identifier(table_name))
                        #NOTE: sql.SQL is needed to specify this parameter as table name (can't be passed as execute second parameter)
                    )

                    printSkippedColumns = True

                    for row in cur.fetchall(): 
                        for column in range(1,len(row)):
                            column_name = cur.description[column][0] 

                            if column_name not in self.embeddingModel or column_name=='id':
                                if printSkippedColumns:
                                    print('\tCOLUMN ',column_name,' SKIPPED')
                                continue

                            ctid = row[0]

                            for word in [word.strip(string.punctuation) for word in str(row[column]).lower().split()]:

                                #Ignoring STOPWORDS
                                if word in stw_set:
                                    continue

                                yield table_name,ctid,column_name, word

                        printSkippedColumns=False

[nltk_data] Downloading package stopwords to /home/paulo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
import psycopg2
from psycopg2 import sql
import string

import nltk 
from nltk.corpus import stopwords
nltk.download('stopwords')

stw_set = set(stopwords.words('english')) - {'will'}

def createInvertedIndex(embeddingModel,dbname='dblp',user='imdb',password='imdb',showLog=True):
    #Output: wordHash (Term Index) with this structure below
    #map['word'] = [ 'table': ( {column} , ['ctid'] ) ]

    '''
    The Term Index is built in a preprocessing step that scans only
    once all the relations over which the queries will be issued.
    '''

    
    wh = WordHash()
    ah = {}
    
    previousTable = None
    
    for table,ctid,column,word in DatabaseIter(model):
        
        ah.setdefault(table,{}).setdefault(column,set()).add(word)
        
        wh.addMapping(word,table,column,ctid)
        
    for table in ah:
        for (column, word_set) in ah[table].items():
            num_distinct_words = len(word_set)
            norm = 0
            word_set.clear()
            ah[table][column] = (norm,num_distinct_words)

    print ('INVERTED INDEX CREATED')
    gc.collect()
    return wh,ah

[nltk_data] Downloading package stopwords to /home/paulo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
wh,ah = createInvertedIndex(model)

INDEXING TABLE  paper
	COLUMN  paper_key  SKIPPED
INDEXING TABLE  citation
	COLUMN  paper_cite_key  SKIPPED
	COLUMN  paper_cited_key  SKIPPED
INDEXING TABLE  author
	COLUMN  paper_key  SKIPPED
INDEXING TABLE  conference
	COLUMN  conf_key  SKIPPED
INVERTED INDEX CREATED


In [9]:
from math import log1p 

def processIAF(wordHash,attributeHash):
    
    total_attributes = sum([len(attribute) for attribute in attributeHash.values()])
    
    for (term, values) in wordHash.items():
        attributes_with_this_term = sum([len(attribute) for attribute in wordHash[term].values()])
        IAF = log1p(total_attributes/attributes_with_this_term)
        wordHash.setIAF(term,IAF)
        
    print('IAF PROCESSED')

In [10]:
processIAF(wh,ah)

IAF PROCESSED


In [11]:
def processNormsOfAttributes(wordHash,attributeHash):    
    for word in wh:
        for table in wh[word]:
            for column, ctids in wh[word][table].items():
                   
                (prevNorm,num_distinct_words)=attributeHash[table][column]

                IAF = wordHash.getIAF(word)

                TF = len(ctids)
                
                Norm = prevNorm + (TF*IAF)

                attributeHash[table][column]=(Norm,num_distinct_words)
                
    print ('NORMS OF ATTRIBUTES PROCESSED')

In [12]:
processNormsOfAttributes(wh,ah)

NORMS OF ATTRIBUTES PROCESSED


In [13]:
ah

{'author': {'name': (25454301.212786403, 535435)},
 'conference': {'detail': (582255.903084951, 20340),
  'name': (60830.038091471186, 9438)},
 'paper': {'conference': (2974956.5728689986, 8660),
  'title': (20413707.221213866, 487592),
  'year': (1973389.3829474119, 60)}}

In [277]:
import copy
class Tupleset:
   
    def __init__(self, table, predicates = None, tuples = None):            
        
        self.table = table
        self.predicates= predicates if predicates is not None else {}
        self.tuples= tuples if tuples is not None else set()
        
    def addTuple(self, tuple_id):
        self.tuples.add(tuple_id)
        
    def addTuples(self, tuple_ids):
        self.tuples.update(tuple_ids)
        
    def addAttribute(self,attribute):
        self.attributes[attribute].setdefault( (set(),set()) )
    
    def union(self, otherTupleset, changeSources = False):
              
        if self.table != otherTupleset.table:
            return None
        
        if self.table == None:
            return None
        
        if len(self.getKeywords() & otherTupleset.getKeywords())>0:
#             print('tuple sets com palavras repetidas {} \n {} \n ******************'.format(self,otherTupleset))
            return None
        
        jointTuples = self.tuples & otherTupleset.tuples
        
        jointPredicates = {}
        
        jointPredicates.update(copy.deepcopy(self.predicates))
        
        for attribute, (schemaWords, valueWords) in otherTupleset.predicates.items():  
            jointPredicates.setdefault(attribute,   (set(),set())    ) 
            jointPredicates[attribute][0].update(schemaWords)
            jointPredicates[attribute][1].update(valueWords)
            
        jointTupleset = Tupleset(self.table, jointPredicates , jointTuples)
        
        if changeSources:
            self.tuples.difference_update(jointTuples)
            otherTupleset.tuples.difference_update(jointTuples)
        
        return jointTupleset    
        
    def addValueMapping(self,attribute,valueWord):
        self.predicates.setdefault(attribute,   (set(),set())    ) 
        self.predicates[attribute][1].add(valueWord)
        
    
    def addSchemaMapping(self,attribute,schemaWord):
        self.predicates.setdefault(attribute,   (set(),set())    ) 
        self.predicates[attribute][0].add(schemaWord)
        
        
#     def addValueMappings(self,attribute,valueWords):
#         self.predicates.setdefault(attribute,   (set(),set())    ) 
#         self.predicates[attribute][1].update(valueWords)
        
    def getKeywords(self):
        keywords = set()
        for attribute in self.predicates.keys():
            
            schemaWords,valueWords = self.predicates[attribute]
            
            keywords.update(schemaWords)                      
            keywords.update(valueWords)
        return frozenset(keywords)
        
    def isFreeTupleset(self):
        return len(self.predicates)==0
    
    def hasTuples(self):
        return len(self.tuples)>0
    
    def clearTuples(self):
        self.tuples.clear()
    
    def __repr__(self):
        return self.__str__()
    
    def __str__(self):
        result = self.table.upper()
        str_predicates = []
        
        for attribute in self.predicates.keys():
            schemaWords , valueWords = self.predicates[attribute]
            
            if schemaWords == set():
                schemaWords = {}
                
            if valueWords == set():
                valueWords = {}
            
            
            str_predicates.append (attribute + str(schemaWords) + str(valueWords))
            
        result += "(" + ','.join(str_predicates) + ")"
        return result
        
    
        
x = Tupleset('paper')
x.addValueMapping('title','discover')
x.addTuple(1)
x.addTuple(2)

y = Tupleset('paper')
y.addValueMapping('title','2002')
y.addTuple(1)
y.addTuple(3)

w = x.union(y,changeSources = True)
pp([x,y,w])

[PAPER(title{}{'discover'}),
 PAPER(title{}{'2002'}),
 PAPER(title{}{'2002', 'discover'})]


In [278]:
import itertools
def TSFindClass(Q,wordHash):
    #Input:  A keyword query Q=[k1, k2, . . . , km]
    #Output: Set of non-free and non-empty tuple-sets Rq

    '''
    The tuple-set Rki contains the tuples of Ri that contain all
    terms of K and no other keywords from Q
    '''
    
    #Part 1: Find sets of tuples containing each keyword
    P = []
    for keyword in Q:
        
        if keyword not in wordHash:
            continue
        
        for table in wordHash[keyword]:
            for (attribute,ctids) in wordHash[keyword][table].items():
                
                ts = Tupleset(table)
                ts.addValueMapping(attribute,keyword)
                ts.addTuples(ctids)                
                P.append(ts)
    
    #Part 2: Find sets of tuples containing larger termsets
    TSInterMartins(P)
    
    
    #Part 3: Clean tuples
    for ts in P:
        ts.clearTuples()
    
    
    return P

# def TSInter(P):
#     #Input: A Set of non-empty tuple-sets for each keyword alone P 
#     #Output: The Set P, but now including larger termsets (process Intersections)

    
    
#     '''
#     Termset is any non-empty subset K of the terms of a query Q        
#     '''
    
#     Pprev = {}
#     Pprev=copy.deepcopy(P)
#     Pcurr = {}

#     combinations = [x for x in itertools.combinations(Pprev.keys(),2)]
#     for ( Ki , Kj ) in combinations:
#         Tki = Pprev[Ki]
#         Tkj = Pprev[Kj]
        
#         X = Ki | Kj
#         Tx = Tki & Tkj        
        
#         if len(Tx) > 0:            
#             Pcurr[X]  = Tx            
#             Pprev[Ki] = Tki - Tx         
#             Pprev[Kj] = Tkj - Tx
            
#     if Pcurr != {}:
#         Pcurr = copy.deepcopy(TSInter(Pcurr))
        
#     #Pprev = Pprev U Pcurr
#     Pprev.update(Pcurr)     
#     return Pprev   


def TSInterMartins(P):
    #Input: A Set of non-empty tuple-sets for each keyword alone P 
    #Output: The Set P, but now including larger termsets (process Intersections)

    '''
    Termset is any non-empty subset K of the terms of a query Q        
    '''
    
#     print('TSInter\n')
#     pp(P)
#     print('\n====================================\n')
    
    somethingChanged = False
    
    combinations = [x for x in itertools.combinations(P,2)]
    for ( Ti , Tj ) in combinations:
        
#         print('\nTESTANDO UNION {} \n {} \n'.format(Ti,Tj))
        
        
#         print('´´´´´´´TSInter\n')
#         pp(P)
        
        Tx = Ti.union(Tj, changeSources = True)        
        
#         print('\nUNION COMPILADO de {} \n {} \n {}\n\n\n'.format(Ti,Tj,Tx))
        
#         if Tx is not None:
#             print(len(Tx.tuples), 'tuples on union')
            
#         print('´´´´´´´TSInter\n')
#         pp(P)    
        
        
        if Tx is not None and Tx.hasTuples():            
            P.append(Tx)
            
            if Ti.hasTuples() == False:
#                 print('Ti {} has not tuples',Ti)
                P.remove(Ti)
#             else:
#                 print('{} has {} tuples'.format(Ti,len(Ti.tuples)))
                
            if Tj.hasTuples() == False:
#                 print('Tj {} has not tuples',Tj)
                P.remove(Tj)
#             else:
#                 print('{} has {} tuples'.format(Tj,len(Tj.tuples)))
            
            somethingChanged = True
            
    if somethingChanged:
        TSInterMartins(P)

In [279]:
def getQuerySets(filename='querysets/queryset_dblp_martins.txt'):
    QuerySet = []
    with open(filename,encoding='utf-8-sig') as f:
        for line in f.readlines():
            
            #The line bellow Remove words not in OLIVEIRA experiments
            #Q = [word.strip(string.punctuation) for word in line.split() if word not in ['title','dr.',"here's",'char','name'] and word not in stw_set]  
            
            Q = tuple([word.strip(string.punctuation) for word in line.lower().split() if word not in stw_set])
            
            QuerySet.append(Q)
    return QuerySet

In [280]:
Q= ['datacenter','2015']

In [281]:
Rq = TSFindClass(Q,wh)

In [282]:
Rq

[PAPER(title{}{'datacenter'}),
 PAPER(title{}{'2015'}),
 PAPER(year{}{'2015'}),
 CONFERENCE(name{}{'2015'}),
 CONFERENCE(detail{}{'2015'}),
 PAPER(title{}{'datacenter'},year{}{'2015'})]

In [283]:
class SchemaGraph:
    
    def __init__(self):
        self.__graph = {}
    
    def addRelationship(self,tableA,columnA,tableB, columnB, direction = 1):
        
        #A->B
        edge_info = (columnA,columnB,direction)
        self.__graph.setdefault(tableA,{}).setdefault(tableB,[]).append(edge_info)
        
        #B<-A
        edge_info = (columnB,columnA,direction*-1)
        self.__graph.setdefault(tableB,{}).setdefault(tableA,[]).append(edge_info)
        
    def tables(self):
        return self.__graph.keys()
        
    def getAdjacentTables(self, table):
        return self.__graph[table].keys()
        
    def __repr__(self):
        return repr(self.__graph)
    
    def __str__(self):
        return repr(self.__graph)

In [284]:
def getSchemaGraph(dbname='dblp',user='imdb',password='imdb'):
    #Output: A Schema Graph G  with the structure below:
    # G['node'] = edges
    # G['table'] = { 'foreign_table' : (direction, column, foreign_column) }
    
    G = SchemaGraph()
    with psycopg2.connect(dbname=dbname,user=user,password=password) as conn:
            with conn.cursor() as cur:
                sql = "SELECT DISTINCT                 tc.table_name, kcu.column_name,                 ccu.table_name AS foreign_table_name, ccu.column_name AS foreign_column_name             FROM information_schema.table_constraints AS tc              JOIN information_schema.key_column_usage AS kcu                 ON tc.constraint_name = kcu.constraint_name             JOIN information_schema.constraint_column_usage AS ccu                 ON ccu.constraint_name = tc.constraint_name             WHERE constraint_type = 'FOREIGN KEY'"
                cur.execute(sql)
                relations = cur.fetchall()

                for (table,column,foreign_table,foreign_column) in relations:
                    print('table,column,foreign_table,foreign_column\n{}, {}, {}, {}'.format(table,column,foreign_table,foreign_column))
                    G.addRelationship(table,column,foreign_table,foreign_column)  
                print ('SCHEMA CREATED')          
    return G

In [285]:
G=getSchemaGraph()

table,column,foreign_table,foreign_column
author, paper_key, paper, paper_key
table,column,foreign_table,foreign_column
citation, paper_cite_key, paper, paper_key
table,column,foreign_table,foreign_column
citation, paper_cited_key, paper, paper_key
SCHEMA CREATED


In [304]:
import copy
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn

class Similarities:
    
    def __init__(self, model, attributeHash,schemaGraph):

        self.model = model
        self.attributeHash = attributeHash
        self.schemaGraph = schemaGraph
        
        self.loadEmbeddingHashes()
    
    
    def wordnet_similarity(self,wordA,wordB):
        A = set(wn.synsets(wordA))
        B = set(wn.synsets(wordB))

        wupSimilarities = [0]
        pathSimilarities = [0]
        
        for (sense1,sense2) in itertools.product(A,B):        
            wupSimilarities.append(wn.wup_similarity(sense1,sense2) or 0)
            pathSimilarities.append(wn.path_similarity(sense1,sense2) or 0)
            
        return max(max(wupSimilarities),max(pathSimilarities))

    def jaccard_similarity(self,wordA,wordB):

        A = set(wordA)
        B = set(wordB)

        return len(A & B ) / len(A | B)
    
    
    def embedding10_similarity(self,word,table,column='*',Emb='B'):
        wnl = WordNetLemmatizer()
        
        # Os sinônimos do EmbA também são utilizados por todos
        sim_list = self.EmbA[table][column]
        
        if column != '*':
        
            if Emb == 'B':
                sim_list |= self.EmbB[table][column]

            elif Emb == 'C':
                sim_list |= self.EmbC[table][column]

        return wnl.lemmatize(word) in sim_list
    
    
    def embedding_similarity(self,wordA,wordB):
        if wordA not in self.model or wordB not in self.model:
            return 0
        return self.model.similarity(wordA,wordB)
    
    
    def word_similarity(self,word,table,column = '*',
                    wn_sim=True, 
                    jaccard_sim=True,
                    emb_sim=False,
                    emb10_sim='B'):
        sim_list=[0]
    
        if column == '*':
            schema_term = table
        else:
            schema_term = column

        if wn_sim:
            sim_list.append( self.wordnet_similarity(schema_term,word) )

        if jaccard_sim:
            sim_list.append( self.jaccard_similarity(schema_term,word) )

        if emb_sim:
            sim_list.append( self.embedding_similarity(schema_term,word) )

        sim = max(sim_list) 

        if emb10_sim:
            if self.embedding10_similarity(word,table,column,emb10_sim):
                if len(sim_list)==1:
                    sim=1
            else:
                sim=0
                
        print('sim({},{}.{}) = {}'.format(word,table,column,sim))        
        
        return sim    
    
    def __getSimilarSet(self,word, inputType = 'word'):
        if inputType == 'vector':
            sim_list = model.similar_by_vector(word)
        else:
            sim_list = model.most_similar(word)        
        return  {word.lower() for word,sim in sim_list}
    
    def loadEmbeddingHashes(self,weight=0.5):
        
        self.EmbA = {}
        self.EmbB = {}
        self.EmbC = {}
    
        for table in self.attributeHash:

            if table not in self.model:
                continue

            self.EmbA[table]={}
            self.EmbB[table]= {}
            self.EmbC[table]= {}
            
            self.EmbA[table]['*'] = self.__getSimilarSet(table) 

            for column in self.attributeHash[table]:
                if column not in model or column=='id':
                    continue
                
                self.EmbA[table][column]=self.__getSimilarSet(column)
                
                self.EmbB[table][column]=self.__getSimilarSet( (table,column) )
                  
                avg_vec = (model[table]*weight + model[column]*(1-weight))                   
                self.EmbC[table][column] = self.__getSimilarSet(avg_vec, inputType = 'vector')
                
                
                
        G = self.schemaGraph
        for tableA in G.tables():

            if tableA not in self.attributeHash or tableA not in model:
                continue

            for tableB in G.getAdjacentTables(tableA):

                if tableB not in self.attributeHash or tableB not in model:
                    continue

                self.EmbB[tableB][tableA] = self.EmbB[tableA][tableB] = self.__getSimilarSet( (tableA,tableB) )

        

In [309]:
def SchSFind(Q,attributeHash,threshold=0.8, 
             sim_args={}):    
    S = []
    
    sm = Similarities(model,ah,G)
    
    for keyword in Q:
        for (table,values) in attributeHash.items():
            
            sim = sm.word_similarity(keyword,table,**sim_args)
            
            if sim >= threshold:  
                ts = Tupleset(table)
                ts.addSchemaMapping('*',keyword)
                S.append(ts)
            
            for attribute in values.keys():
                
                if(attribute=='id'):
                    continue
                
                sim = sm.word_similarity(keyword,table,attribute,**sim_args)
                
                if sim >= threshold:
                    ts = Tupleset(table)
                    ts.addSchemaMapping(attribute,keyword)
                    S.append(ts)
                    
    return S

In [310]:
Q = ['author','name']

In [311]:
Sq = SchSFind(Q,ah)

sim(author,paper.*) = 0
sim(author,paper.title) = 0
sim(author,paper.year) = 0
sim(author,paper.conference) = 0
sim(author,conference.*) = 0
sim(author,conference.detail) = 0
sim(author,conference.name) = 0
sim(author,author.*) = 1.0
sim(author,author.name) = 0.631578947368421
sim(name,paper.*) = 0
sim(name,paper.title) = 0
sim(name,paper.year) = 0
sim(name,paper.conference) = 0
sim(name,conference.*) = 0
sim(name,conference.detail) = 0
sim(name,conference.name) = 1.0
sim(name,author.*) = 0
sim(name,author.name) = 1.0


In [308]:
Sq

set()